In [1]:
import psycopg2
import os
from dotenv import load_dotenv

load_dotenv()

conn = psycopg2.connect(
    host=os.getenv("POSTGRES_SERVER").split(":")[0],
    port=os.getenv("POSTGRES_SERVER").split(":")[1],
    user=os.getenv("POSTGRES_USERNAME"),
    password=os.getenv("POSTGRES_PASSWORD"),
    database=os.getenv("POSTGRES_DATABASE")
)

cur = conn.cursor()
cur.execute("SELECT * FROM patients LIMIT 5;")
rows = cur.fetchall()

rows

[('FB2ABB23-C9D0-4D09-8464-49BF0B982F0F',
  2,
  datetime.datetime(1947, 12, 28, 2, 45, 40, 547000),
  3,
  3,
  3,
  18.08),
 ('64182B95-EB72-4E2B-BE77-8050B71498CE',
  2,
  datetime.datetime(1952, 1, 18, 19, 51, 12, 917000),
  4,
  6,
  4,
  13.03),
 ('DB22A4D9-7E4D-485C-916A-9CD1386507FB',
  1,
  datetime.datetime(1970, 7, 25, 13, 4, 20, 717000),
  2,
  3,
  4,
  6.67),
 ('6E70D84D-C75F-477C-BC37-9177C3698C66',
  2,
  datetime.datetime(1979, 1, 4, 5, 45, 29, 580000),
  1,
  3,
  4,
  16.09),
 ('C8556CC0-32FC-4CA5-A8CD-9CCF38816167',
  1,
  datetime.datetime(1921, 4, 11, 11, 39, 49, 197000),
  1,
  3,
  4,
  18.2)]